## NHS Master Data Managment (in England)

It is quite common to load in master data files from [ODS Data Search and Export](https://digital.nhs.uk/services/organisation-data-service/data-search-and-export/csv-downloads) into a computer system. The identifiers used for GP's and Organisations help identify these entities between different systems and health providers.

NHS England provides several API's for doing this:

- [Organisation Data Terminology - FHIR API](https://digital.nhs.uk/developer/api-catalogue/organisation-data-terminology) which allows you to search for organisations
- [Spine Directory Service - LDAP API](https://digital.nhs.uk/developer/api-catalogue/spine-directory-service-ldap) which allows search on a wide set of MDM entities and includes most of the entities from ODS.

The structure of these entities in FHIR, ODS and SDS is very similar. This diagram is from [HL7 FHIR Administration Module](https://hl7.org/fhir/R4/administration-module.html)

![Alt text](https://hl7.org/fhir/R4/administration-module-prov-dir.png)

### Care Directory Service

In this guide we are aiming to produce a FHIR API following [IHE Mobile Care Services Discovery (mCSD)](https://profiles.ihe.net/ITI/mCSD/index.html). We won't get a to complete implementation as the health services are available in a variety of `directory of services` APIs, such as:

- [Directory of Healthcare Services (Service Search) API](https://digital.nhs.uk/developer/api-catalogue/directory-of-healthcare-services)
- [Electronic Transmission of Prescriptions Web Services - SOAP API](https://digital.nhs.uk/developer/api-catalogue/electronic-transmission-of-prescriptions-web-services-soap)

### Plan/Design

This notebook explores the ETL process in the diagram below.

![Alt text](images/ETL+Airflow.drawio.png)



## Organisation (epraccur)

### Extract - Import from NHS England Organisation Data Services

In [234]:
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd

maxentries = 10000

headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}
odsurl = 'https://files.digital.nhs.uk/assets/ods/current/epraccur.zip'
response = requests.get(odsurl, headers=headers, timeout=120)
response.raise_for_status()  # Raise an exception for bad status codes

myzip = ZipFile(BytesIO(response.content))

with myzip.open('epraccur.csv') as csv_file:
    epraccur = pd.read_csv(
        csv_file,
        header=None,
        index_col=False,
        names=[
            "ODS", "Organisation_Name", "NationalGrouping", 4,
            "AddressLine_1", "AddressLine_2", "AddressLine_3",
            "AddressLine_4", "AddressLine_5", "PostCode",
            "Opened", "Closed", 13, 14, "PRAC_ODS",
            16, 17, "PhoneNumber", 19, 20, 21, 22, 23, 24, 25, 26
        ]
    )


epraccur = epraccur.set_index(['ODS'])
epraccur

,Organisation_Name,NationalGrouping,4,AddressLine_1,AddressLine_2,AddressLine_3,AddressLine_4,AddressLine_5,PostCode,Opened,...,17,PhoneNumber,19,20,21,22,23,24,25,26
ODS,,,,,,,,,,,,,,,,,,,,,
A81001,THE DENSHAM SURGERY,Y63,QHM,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,NaN,TS18 1HU,19740401,...,NaN,01642 672351,NaN,NaN,NaN,0,NaN,16C,NaN,4
A81002,QUEENS PARK MEDICAL CENTRE,Y63,QHM,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,NaN,TS18 2AW,19740401,...,NaN,01642 618170,NaN,NaN,NaN,0,NaN,16C,NaN,4
A81003,VICTORIA MEDICAL PRACTICE,Y54,Q74,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,NaN,TS26 8DB,19740401,...,20171031.0,01429 272945,NaN,NaN,NaN,0,NaN,00K,NaN,4
A81004,ACKLAM MEDICAL CENTRE,Y63,QHM,TRIMDON AVENUE,ACKLAM,MIDDLESBROUGH,CLEVELAND,NaN,TS5 8SB,19740401,...,NaN,01642 827697,NaN,NaN,NaN,0,NaN,16C,NaN,4
A81005,SPRINGWOOD SURGERY,Y63,QHM,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,NaN,TS14 7DJ,19740401,...,NaN,01287 619611,NaN,NaN,NaN,0,NaN,16C,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y08757,COMMUNITY HOSPITAL ALCOHOL TEAM,Y60,QNC,EDWARD MYERS UNIT,HARPLANDS HOSPITAL,STOKE-ON-TRENT,STAFFORDSHIRE,NaN,ST4 6TH,20250501,...,NaN,01782 441715,NaN,NaN,NaN,1,NaN,RLY,NaN,10
Y08758,LARC SERVICE,Y60,QJM,COUNTY OFFICES,NEWLAND,LINCOLN,LINCOLNSHIRE,NaN,LN1 1YL,20250401,...,NaN,01522 554980,NaN,NaN,NaN,1,NaN,503,NaN,8
Y08759,WELL LIFE CLINIC,Y59,QXU,THE HOUSE PARTNERSHIP,99 STATION ROAD,REDHILL,SURREY,NaN,RH1 1EB,20250601,...,NaN,01737 761201,NaN,NaN,NaN,1,NaN,92A,NaN,0


### Extract - Existing Organisations from Repository

#### Demonstration FHIR Server and Database

The examples that follow use an Intersystems FHIR Repository. Instructions for running this on a local machine can be found here [ris-fhirserver-template](https://github.com/intersystems-community/iris-fhir-template/blob/master/README.md)

Once installed, you can browse to the [SQL Explorer](http://localhost:32783/csp/sys/exp/%25CSP.UI.Portal.SQL.Home.zen?$NAMESPACE=FHIRSERVER) - username is _System and password SYS

Then execute the following SQL.

`select * from HSFHIR_X0001_S.Organization where addressCountry <> 'US'`

Note the IRIS demo comes with some preloaded test data; the where clause excludes this. Python version is below:

In [235]:
import iris
import pandas as pd


host = "localhost"
# this is the superserver port
port = 32782
namespace = "FHIRSERVER"
user = "_SYSTEM"
password = "SYS"

conn = iris.connect(
    hostname=host,
    port=port,
    namespace=namespace,
    username=user,
    password=password
)

# create a cursor
cursor = conn.cursor()

sql = """
      select org._id, org.Key, org.Identifier, org._lastUpdated, resource.ResourceString, null as ODS from HSFHIR_X0001_S.Organization org
                                                                                                               join HSFHIR_X0001_R.Rsrc resource on resource.Key = org.Key
      where IsNull(org.addressCountry,'') <> 'US' and org.type [ 'https://fhir.nhs.uk/CodeSystem/organisation-role|76'
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
pd.set_option('future.no_silent_downcasting', True)
df

,_id,Key,identifier,_lastUpdated,ResourceString,ODS
0,680154,Organization/680154,"A81001,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{...",None
1,680155,Organization/680155,"A81002,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{...",None
2,680156,Organization/680156,"A81003,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{...",None
3,680157,Organization/680157,"A81004,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{...",None
4,680158,Organization/680158,"A81005,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{...",None
...,...,...,...,...,...,...
9995,697426,Organization/697426,"Y01699,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T11:02:33Z,"{""resourceType"":""Organization"",""identifier"":[{...",None
9996,697427,Organization/697427,"Y01700,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T11:02:33Z,"{""resourceType"":""Organization"",""identifier"":[{...",None
9997,697428,Organization/697428,"Y01701,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T11:02:33Z,"{""resourceType"":""Organization"",""identifier"":[{...",None
9998,697429,Organization/697429,"Y01704,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T11:02:33Z,"{""resourceType"":""Organization"",""identifier"":[{...",None


Merge the two datasets

In [236]:
import re
for orgId in range(0, len(df)):
    identifier = df.loc[orgId,'identifier']
    identifiers = identifier.split(',')
    for id in identifiers:
        if (re.match('^https:.*ods-organization-code[|][A-Za-z0-9].*$',id)):
            df.loc[orgId,'ODS'] = id.split('|')[1]


organisations = pd.merge(epraccur, df, how="left", on=["ODS"])
organisations = organisations.set_index(['ODS'])

organisations['_id'] = organisations['_id'].fillna(-1).astype(int)
organisations

,Organisation_Name,NationalGrouping,4,AddressLine_1,AddressLine_2,AddressLine_3,AddressLine_4,AddressLine_5,PostCode,Opened,...,22,23,24,25,26,_id,Key,identifier,_lastUpdated,ResourceString
ODS,,,,,,,,,,,,,,,,,,,,,
A81001,THE DENSHAM SURGERY,Y63,QHM,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,NaN,TS18 1HU,19740401,...,0,NaN,16C,NaN,4,680154,Organization/680154,"A81001,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{..."
A81002,QUEENS PARK MEDICAL CENTRE,Y63,QHM,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,NaN,TS18 2AW,19740401,...,0,NaN,16C,NaN,4,680155,Organization/680155,"A81002,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{..."
A81003,VICTORIA MEDICAL PRACTICE,Y54,Q74,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,NaN,TS26 8DB,19740401,...,0,NaN,00K,NaN,4,680156,Organization/680156,"A81003,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{..."
A81004,ACKLAM MEDICAL CENTRE,Y63,QHM,TRIMDON AVENUE,ACKLAM,MIDDLESBROUGH,CLEVELAND,NaN,TS5 8SB,19740401,...,0,NaN,16C,NaN,4,680157,Organization/680157,"A81004,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{..."
A81005,SPRINGWOOD SURGERY,Y63,QHM,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,NaN,TS14 7DJ,19740401,...,0,NaN,16C,NaN,4,680158,Organization/680158,"A81005,https://fhir.nhs.uk/Id/ods-organization...",2025-06-16T09:34:14Z,"{""resourceType"":""Organization"",""identifier"":[{..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y08757,COMMUNITY HOSPITAL ALCOHOL TEAM,Y60,QNC,EDWARD MYERS UNIT,HARPLANDS HOSPITAL,STOKE-ON-TRENT,STAFFORDSHIRE,NaN,ST4 6TH,20250501,...,1,NaN,RLY,NaN,10,-1,NaN,NaN,NaN,NaN
Y08758,LARC SERVICE,Y60,QJM,COUNTY OFFICES,NEWLAND,LINCOLN,LINCOLNSHIRE,NaN,LN1 1YL,20250401,...,1,NaN,503,NaN,8,-1,NaN,NaN,NaN,NaN
Y08759,WELL LIFE CLINIC,Y59,QXU,THE HOUSE PARTNERSHIP,99 STATION ROAD,REDHILL,SURREY,NaN,RH1 1EB,20250601,...,1,NaN,92A,NaN,0,-1,NaN,NaN,NaN,NaN


### Transform - FHIR Organization Function

In [237]:
from fhir.resources.R4B.organization import Organization


def convertOrganisationFHIR(org):
    organisationJSON = {
        "resourceType": "Organization",
        "identifier": [
            {
                "system": "https://fhir.nhs.uk/Id/ods-organization-code",
                "value": org
            }
        ],
        "active": True,
        "type": [
            {
                "coding": [
                    {
                        "system": "https://fhir.nhs.uk/CodeSystem/organisation-role",
                        "code": "76",
                        "display": "GP PRACTICE"
                    }
                ]
            }
        ],
        "name": organisations.loc[org,'Organisation_Name']
    }
    # if org is closed update active field
    #
    #
    if (organisations.loc[org,'PostCode'] != '' and not pd.isnull(organisations.loc[org,'PostCode'])):
        organisationJSON["address"]: [
            {
                "use": "work",
                "postalCode": organisations.loc[org,'PostCode']
            }
        ]
    if (organisations.loc[org,'NationalGrouping'] != '' and not pd.isnull(organisations.loc[org,'NationalGrouping'])):
        organisationJSON["partOf"] = {
            "identifier": {
                "system": "https://fhir.nhs.uk/Id/ods-organization-code",
                "value": organisations.loc[org,'NationalGrouping']
            }
        }
    if (organisations.loc[org,'PhoneNumber'] != '' and not pd.isnull(organisations.loc[org,'PhoneNumber'])):
        # print('-',organisations.loc[org,'PhoneNumber'].strip(),'-',"1")
        organisationJSON['telecom'] = [
            {
                "system": "phone",
                "value": organisations.loc[org,'PhoneNumber'].strip(),
                "use": "work"
            }]
    if (organisations.loc[org,'PostCode'] != '' and not pd.isnull(organisations.loc[org,'PostCode'])):
        # print('-',organisations.loc[org,'PhoneNumber'].strip(),'-',"1")
        organisationJSON['address'] = [
            {
                "line": [
                ],
                "postalCode" : organisations.loc[org,'PostCode']
            }]
        if (organisations.loc[org,'AddressLine_1'] != '' and not pd.isnull(organisations.loc[org,'AddressLine_1'])):
            organisationJSON['address'][0]['line'].append(organisations.loc[org,'AddressLine_1'])
        if (organisations.loc[org,'AddressLine_2'] != '' and not pd.isnull(organisations.loc[org,'AddressLine_2'])):
            organisationJSON['address'][0]['line'].append(organisations.loc[org,'AddressLine_2'])
        if (organisations.loc[org,'AddressLine_3'] != '' and not pd.isnull(organisations.loc[org,'AddressLine_3'])):
            organisationJSON['address'][0]['city'] = organisations.loc[org,'AddressLine_3']
        if (organisations.loc[org,'AddressLine_4'] != '' and not pd.isnull(organisations.loc[org,'AddressLine_4'])):
            organisationJSON['address'][0]['district'] = organisations.loc[org,'AddressLine_4']
    if organisations.loc[org,'Closed'] != '' and not pd.isnull(organisations.loc[org,'Closed']) :
        organisationJSON['active'] = False
    if organisations.loc[org,'_id'] != -1:
        organisationJSON['id'] = str(organisations.loc[org,'_id'])
    #print(json.dumps(organisationJSON, indent=2, ensure_ascii=False))
    # validate organisation against schema
    Organization(**organisationJSON)
    return organisationJSON

### Load Organisation - Update FHIR Repository

The results from the above will vary.

What we need to do merge the organisations from ODS and also the organisations in our database.

The outline logic will be:

- Organisation exists in both:
  - ODS will be assumed to be the master record for active and address fields, if ODS has different values then update the database.
  - If our telephone field is empty, then update with ODS entry, otherwise do not process.
- Organisation does not exist in our database.
  - Add the ODS organisation

Firstly, we need to merge the dataframe (df) we have retrieved from the FHIR repository with the epraccur data frame.

We do this on the ODS code which in the df dataframe is an array. This code has a system of `https://fhir.nhs.uk/Id/ods-organization-code`, so we need to use the entry with this and this also has a value.


In [238]:

import json

headers = {"Content-Type": "application/fhir+json"}
fhirurl = "http://localhost:32783/fhir/r4"

new = organisations[['_id']].head(maxentries).copy()
for org in new.index:

    organisationJSON = convertOrganisationFHIR(org)
    #print(json.dumps(organisationJSON, indent=2, ensure_ascii=False))

    if (organisations.loc[org,'_id'] == -1):

        # Create

        r = requests.post(fhirurl+'/Organization', data=json.dumps(organisationJSON), headers=headers)

        if 'Location' in r.headers:
            location = r.headers['Location'].split('Organization/')[1].split('/')[0]
            organisations.loc[org,'_id'] = str(location)
            print('Created ' + org + ' id - '+location)
        else:
            print("No Location header in response: ",r.status_code)
            print("Response headers:", r.headers)
            print(json.dumps(organisationJSON, indent=2, ensure_ascii=False))
            print(r.text)
    else:
        # Update

        isUpdate = False
        if organisations.loc[org,'ResourceString'] != '':
            organisationMDM = json.loads(organisations.loc[org,'ResourceString'])
            #organisationMDM['name'] = organisations.loc[org,'Organisation_Name']
            if ('telecom' in organisationMDM and
                    isinstance(organisationMDM['telecom'], list) and
                    len(organisationMDM['telecom']) > 0 and
                    organisationMDM['telecom'][0].get('value', '') == ''):
                isUpdate = True
                organisationMDM['telecom'] = organisationJSON['telecom']
                print('telephone')
            if ('address' in organisationMDM and
                    isinstance(organisationMDM['address'], list) and
                    len(organisationMDM['address']) > 0 and
                    organisationMDM['address'][0].get('postalCode', '') != organisations.loc[org,'PostCode']):
                isUpdate = True
                print('postcode')
                organisationMDM['address'] = organisationJSON['address']
            if ('address' not in organisationMDM) and not pd.isnull(organisations.loc[org,'PostCode']):
                isUpdate = True
                print('postcode ',organisations.loc[org,'PostCode'])
                organisationMDM['address'] = organisationJSON['address']
            if organisationMDM.get('partOf', {}).get('identifier', {}).get('value', '') != organisations.loc[org,'NationalGrouping'] and not pd.isnull(organisations.loc[org,'NationalGrouping']):
                isUpdate = True
                organisationMDM['partOf'] = organisationJSON['partOf']
                print('partOf - ',organisations.loc[org,'NationalGrouping'])

            if organisationMDM['active'] != organisationJSON['active']:
                isUpdate = True
                organisationMDM['active'] = organisationJSON['active']
                print('active')

            if isUpdate:
                #print(json.dumps(organisationMDM, indent=2, ensure_ascii=False))
                r = requests.put(fhirurl+'/Organization/'+organisationMDM['id'], data=json.dumps(organisationMDM), headers=headers)

                if 'Location' in r.headers:
                    location = r.headers['Location'].split('Organization/')[1].split('/')[0]
                    organisations.loc[org,'ResourceString'] = json.dumps(organisationMDM)
                    print('Created ' + org + ' id - '+location)
                else:
                    print("No Location header in response: ",r.status_code)
                    print("Response headers:", r.headers)
                    print(json.dumps(organisationJSON, indent=2, ensure_ascii=False))
                    print(r.text)

## Practitioner

### Extract - Practitioners (egpcur)

The general idea behind this is we want to be able to do some basic queries on ODS data. For example, we may want a list of GPs who work at

In [239]:
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd


headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

zipurl = 'https://files.digital.nhs.uk/assets/ods/current/egpcur.zip'
response = requests.get(zipurl, headers=headers, timeout=120)
response.raise_for_status()  # Raise an exception for bad status codes

myzip = ZipFile(BytesIO(response.content))


with myzip.open('egpcur.csv') as csv_file:
    egpcur = pd.read_csv(
        csv_file,
        header=None,
        index_col=False,
        names=[
            "GMP","Practitioner_Name",3,4,"AddressLine_1","AddressLine_2","AddressLine_3","AddressLine_4","AddressLine_5","PostCode","Started","Ended","Status",13,"ODS",15,16,"PhoneNumber",18,19,20,21,22,23,24,25,26
        ]
    )


C:\Users\lilyh\AppData\Local\Temp\ipykernel_27204\1436373078.py:17: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  egpcur = pd.read_csv(


This next section of code:
- Adds practice name to the GP data frame
- splits the name into surname and initials

In [240]:

#egpcur = pd.merge(egpcur, epraccur['Organisation_Name'], left_on='ODS', right_on='ODS')

egpcur['Practitioner_Surname'] = egpcur['Practitioner_Name'].str.split(' ', expand=True)[0]
egpcur['Practitioner_Initials'] = egpcur['Practitioner_Name'].str.split(' ', expand=True)[1]

egpcur = egpcur.set_index(['GMP'])

egpcur

,Practitioner_Name,3,4,AddressLine_1,AddressLine_2,AddressLine_3,AddressLine_4,AddressLine_5,PostCode,Started,...,19,20,21,22,23,24,25,26,Practitioner_Surname,Practitioner_Initials
GMP,,,,,,,,,,,,,,,,,,,,,
G0102005,ALLEN EB,Y11,QAL,"FIRCROFT, LONDON ROAD",ENGLEFIELD GREEN,EGHAM,SURREY,NaN,TW20 0BS,19740401,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,ALLEN,EB
G0102926,ANDERSON MG,Y61,QUE,LENSFIELD MEDICAL PRAC.,48 LENSFIELD ROAD,CAMBRIDGE,CAMBRIDGESHIRE,NaN,CB2 1EH,19740401,...,NaN,NaN,1,NaN,06H,NaN,NaN,NaN,ANDERSON,MG
G0105912,ADLER S,Y56,QMJ,682 FINCHLEY ROAD,GOLDERS GREEN,LONDON,NaN,NaN,NW11 7NP,19740401,...,NaN,NaN,1,NaN,93C,NaN,NaN,NaN,ADLER,S
G0107031,ATTWOOD DC,Y62,QOP,GREAT LEVER HEALTH CENTRE,"RUPERT STREET,GREAT LEVER",BOLTON,LANCASHIRE,NaN,BL3 6RN,19740401,...,NaN,NaN,1,NaN,00T,NaN,NaN,NaN,ATTWOOD,DC
G0107725,ALEXANDER PJ,Y01,QDF,10 WEST END,SWANLAND,HUMBERSIDE,NaN,NaN,HU14 3PE,19740401,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,ALEXANDER,PJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G9996043,UNIDENTIFIED GPS,W00,Q99,NORTH WALES HA,PRESWYLFA,HENDY ROAD,MOLD FLINTSHIRE,NaN,CH7 1PZ,19740401,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,UNIDENTIFIED,GPS
G9996050,UNIDENTIFIED GPS,W00,Q99,MORGANNWG HA,41 HIGH STREET,SWANSEA,WEST GLAMORGAN,NaN,SA1 1LT,19740401,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,UNIDENTIFIED,GPS
G9996067,COMMITTEES LOCUM,W00,QW3,DEPUTISING SERVICES,POWYS,NaN,NaN,NaN,NaN,19740401,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,COMMITTEES,LOCUM


### Extract - Existing Data

Query the database to get existing entries


In [241]:
sql = """
      select prac._id, prac.Key, prac.Identifier, prac._lastUpdated, resource.ResourceString, null as GMP from HSFHIR_X0001_S.Practitioner prac
                                                                                                                   join HSFHIR_X0001_R.Rsrc resource on  resource.Key = prac.Key
      where IsNull(prac.addressCountry,'') <> 'US'
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

pd.set_option('future.no_silent_downcasting', True)
df

,_id,Key,identifier,_lastUpdated,ResourceString,GMP
0,680160,Practitioner/680160,"G0102005,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None
1,680161,Practitioner/680161,"G0102926,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None
2,680162,Practitioner/680162,"G0105912,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None
3,680163,Practitioner/680163,"G0107031,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None
4,680164,Practitioner/680164,"G0107725,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None
...,...,...,...,...,...,...
9995,702426,Practitioner/702426,"G3430226,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T11:03:35Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None
9996,702427,Practitioner/702427,"G3430264,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T11:03:35Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None
9997,702428,Practitioner/702428,"G3430271,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T11:03:35Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None
9998,702429,Practitioner/702429,"G3430288,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T11:03:35Z,"{""resourceType"":""Practitioner"",""identifier"":[{...",None


## Transform - Merge both of the Practitioner datasets

Extract out the GMP Number and place in a separate field, this field is used for the merge.

In [242]:
for pracId in range(0, len(df)):
    identifier = df.loc[pracId,'identifier']
    identifiers = identifier.split(',')
    for id in identifiers:
        if (re.match('^https:.*gmp-number[|]G[0-9].*$',id)):
            print(id)
            df.loc[pracId,'GMP'] = id.split('|')[1]


practitioners = pd.merge(egpcur, df, how="left", on=["GMP"])
practitioners = practitioners.set_index(['GMP'])
practitioners['_id'] = practitioners['_id'].fillna(-1).astype(int)
practitioners

https://fhir.hl7.org.uk/Id/gmp-number|G0102005
https://fhir.hl7.org.uk/Id/gmp-number|G0102926
https://fhir.hl7.org.uk/Id/gmp-number|G0105912
https://fhir.hl7.org.uk/Id/gmp-number|G0107031
https://fhir.hl7.org.uk/Id/gmp-number|G0107725
https://fhir.hl7.org.uk/Id/gmp-number|G0108018
https://fhir.hl7.org.uk/Id/gmp-number|G0108324
https://fhir.hl7.org.uk/Id/gmp-number|G0108867
https://fhir.hl7.org.uk/Id/gmp-number|G0108922
https://fhir.hl7.org.uk/Id/gmp-number|G0109325
https://fhir.hl7.org.uk/Id/gmp-number|G0109459
https://fhir.hl7.org.uk/Id/gmp-number|G0109497
https://fhir.hl7.org.uk/Id/gmp-number|G0109662
https://fhir.hl7.org.uk/Id/gmp-number|G0109693
https://fhir.hl7.org.uk/Id/gmp-number|G0109741
https://fhir.hl7.org.uk/Id/gmp-number|G0109923
https://fhir.hl7.org.uk/Id/gmp-number|G0110017
https://fhir.hl7.org.uk/Id/gmp-number|G0110031
https://fhir.hl7.org.uk/Id/gmp-number|G0110134
https://fhir.hl7.org.uk/Id/gmp-number|G0110268
https://fhir.hl7.org.uk/Id/gmp-number|G0110464
https://fhir.

,Practitioner_Name,3,4,AddressLine_1,AddressLine_2,AddressLine_3,AddressLine_4,AddressLine_5,PostCode,Started,...,24,25,26,Practitioner_Surname,Practitioner_Initials,_id,Key,identifier,_lastUpdated,ResourceString
GMP,,,,,,,,,,,,,,,,,,,,,
G0102005,ALLEN EB,Y11,QAL,"FIRCROFT, LONDON ROAD",ENGLEFIELD GREEN,EGHAM,SURREY,NaN,TW20 0BS,19740401,...,NaN,NaN,NaN,ALLEN,EB,680160,Practitioner/680160,"G0102005,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{..."
G0102926,ANDERSON MG,Y61,QUE,LENSFIELD MEDICAL PRAC.,48 LENSFIELD ROAD,CAMBRIDGE,CAMBRIDGESHIRE,NaN,CB2 1EH,19740401,...,NaN,NaN,NaN,ANDERSON,MG,680161,Practitioner/680161,"G0102926,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{..."
G0105912,ADLER S,Y56,QMJ,682 FINCHLEY ROAD,GOLDERS GREEN,LONDON,NaN,NaN,NW11 7NP,19740401,...,NaN,NaN,NaN,ADLER,S,680162,Practitioner/680162,"G0105912,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{..."
G0107031,ATTWOOD DC,Y62,QOP,GREAT LEVER HEALTH CENTRE,"RUPERT STREET,GREAT LEVER",BOLTON,LANCASHIRE,NaN,BL3 6RN,19740401,...,NaN,NaN,NaN,ATTWOOD,DC,680163,Practitioner/680163,"G0107031,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{..."
G0107725,ALEXANDER PJ,Y01,QDF,10 WEST END,SWANLAND,HUMBERSIDE,NaN,NaN,HU14 3PE,19740401,...,NaN,NaN,NaN,ALEXANDER,PJ,680164,Practitioner/680164,"G0107725,https://fhir.hl7.org.uk/Id/gmp-number...",2025-06-16T09:47:27Z,"{""resourceType"":""Practitioner"",""identifier"":[{..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G9996043,UNIDENTIFIED GPS,W00,Q99,NORTH WALES HA,PRESWYLFA,HENDY ROAD,MOLD FLINTSHIRE,NaN,CH7 1PZ,19740401,...,NaN,NaN,NaN,UNIDENTIFIED,GPS,-1,NaN,NaN,NaN,NaN
G9996050,UNIDENTIFIED GPS,W00,Q99,MORGANNWG HA,41 HIGH STREET,SWANSEA,WEST GLAMORGAN,NaN,SA1 1LT,19740401,...,NaN,NaN,NaN,UNIDENTIFIED,GPS,-1,NaN,NaN,NaN,NaN
G9996067,COMMITTEES LOCUM,W00,QW3,DEPUTISING SERVICES,POWYS,NaN,NaN,NaN,NaN,19740401,...,NaN,NaN,NaN,COMMITTEES,LOCUM,-1,NaN,NaN,NaN,NaN


### Transform - FHIR Practitioner

In [243]:
from fhir.resources.R4B.practitioner import Practitioner

def convertPractitionerFHIR(GMP):
    practitionerJSON = {
        "resourceType": "Practitioner",
        "identifier": [
            {
                "system": "https://fhir.hl7.org.uk/Id/gmp-number",
                "value": GMP
            }
        ],
        "active": True,
        "name": [
            {
                "family": practitioners.loc[GMP,'Practitioner_Surname'],
                "prefix": [
                    "Dr"
                ]
            }
        ],
        "qualification": [
            {
                "code": {
                    "coding": [ {
                        "system": "http://terminology.hl7.org/CodeSystem/v2-0360",
                        "code": "MD",
                        "display": "Medical Doctor"
                    }
                    ]
                }
            }
        ]
    }
    if practitioners.loc[GMP,'Practitioner_Initials'] != '' and not pd.isnull(practitioners.loc[GMP,'Practitioner_Initials']) :
        practitionerJSON['name'][0]['given'] = [practitioners.loc[GMP,'Practitioner_Initials']]

    if practitioners.loc[GMP,'Status'] != 'C' and not pd.isnull(practitioners.loc[GMP,'Status']) :
        practitionerJSON['active'] = False

    Practitioner(**practitionerJSON)


    return practitionerJSON

#### Example

In [244]:
practitionerDF = egpcur.loc[(egpcur['Practitioner_Surname'] == "KOYA") & (egpcur['Practitioner_Initials'] == "MR")]

GMP = practitionerDF.index[0]

practitionerJSON = convertPractitionerFHIR(GMP)

print(json.dumps(practitionerJSON, indent=2, ensure_ascii=False))

{
  "resourceType": "Practitioner",
  "identifier": [
    {
      "system": "https://fhir.hl7.org.uk/Id/gmp-number",
      "value": "G3298457"
    }
  ],
  "active": false,
  "name": [
    {
      "family": "KOYA",
      "prefix": [
        "Dr"
      ],
      "given": [
        "MR"
      ]
    }
  ],
  "qualification": [
    {
      "code": {
        "coding": [
          {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0360",
            "code": "MD",
            "display": "Medical Doctor"
          }
        ]
      }
    }
  ]
}


### Load

In [245]:
import json

headers = {"Content-Type": "application/fhir+json"}
fhirurl = "http://localhost:32783/fhir/r4"

new = practitioners[['_id']].head(maxentries).copy()
for GMP in new.index:

    practitionerJSON = convertPractitionerFHIR(GMP)
    if (practitioners.loc[GMP,'_id'] == -1):

        # Create

        r = requests.post(fhirurl+'/Practitioner', data=json.dumps(practitionerJSON), headers=headers)

        if 'Location' in r.headers:
            location = r.headers['Location'].split('Practitioner/')[1].split('/')[0]
            practitioners.loc[GMP,'_id'] = str(location)
            print('Created ' + GMP + ' id - '+location)
        else:
            print("No Location header in response: ",r.status_code)
            print("Response headers:", r.headers)
            print(json.dumps(practitionerJSON, indent=2, ensure_ascii=False))
            print(r.text)

## PractitionerRole

A practitioner can work at multiple organisations, so we need a link entity (table).

The element's code and specialty are optional, but we can improve our search capabilities by adding data we can infer from the source file (egpcur). This is the practitioner is a GP and works in General Practice.

Note how we have incorporated identifiers and display names. This is to provide some common data elements in this resource and not require the user to perform another search to retrieve these details, we can clearly see this role is for Dr Koya at the Archway Practice.

### Transform - Practitioner and Organisation into PractitionerRole

In [246]:
# Make GMP column from the index

practitioners['GMP'] = practitioners.index.astype(str)
practitionerroles = pd.merge(practitioners[["GMP","_id","Practitioner_Name","ODS","Started","Ended"]], organisations[["_id","Organisation_Name"]], how="inner", on=["ODS"]).set_index('GMP')

# Remove entries without ID's in the database

practitionerroles = practitionerroles.drop(practitionerroles[practitionerroles._id_x == -1].index)
practitionerroles = practitionerroles.drop(practitionerroles[practitionerroles._id_y == -1].index)

print(practitionerroles.columns)

practitionerroles

Index(['_id_x', 'Practitioner_Name', 'ODS', 'Started', 'Ended', '_id_y',
       'Organisation_Name'],
      dtype='object')


,_id_x,Practitioner_Name,ODS,Started,Ended,_id_y,Organisation_Name
GMP,,,,,,,
G0102926,680161,ANDERSON MG,D81001,19740401,20250514.0,683460,LENSFIELD MEDICAL PRACTICE
G0105912,680162,ADLER S,E83600,19740401,20250514.0,684000,ADLER JS-THE SURGERY
G0107031,680163,ATTWOOD DC,P82013,19740401,20250514.0,695388,LEVER CHAMBERS 2
G0108018,680173,ALLDRIDGE DGE,H81083,19740401,20250514.0,685936,MOAT HOUSE SURGERY
G0108324,680174,ANDERSON CF,A81001,19740401,20250514.0,680154,THE DENSHAM SURGERY
...,...,...,...,...,...,...,...
G3430202,702425,MATHEWS BH,H81072,19740401,NaN,685925,MEDWYN SURGERY
G3430226,702426,ALLISON GE,E81023,19740401,20250514.0,683777,CATER STREET SURGERY
G3430264,702427,LAWTON SL,B86109,19740401,NaN,682628,KIRKSTALL LANE MEDICAL CENTRE


### Extract Existing PractitionerRole from repository

In [247]:
sql = """
      select prac._id, prac.Key, prac._lastUpdated,
             resource.ResourceString, null as ODS, null as GMP
      from HSFHIR_X0001_S.PractitionerRole prac
               inner join HSFHIR_X0001_R.Rsrc resource on resource.Key = prac.Key
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)

pd.set_option('future.no_silent_downcasting', True)
df

,_id,Key,_lastUpdated,ResourceString,ODS,GMP
0,682182,PractitionerRole/682182,2025-06-16T11:21:54Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",None,None
1,682183,PractitionerRole/682183,2025-06-16T11:21:54Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",None,None
2,682184,PractitionerRole/682184,2025-06-16T11:21:54Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",None,None
3,682185,PractitionerRole/682185,2025-06-16T11:21:54Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",None,None
4,682186,PractitionerRole/682186,2025-06-16T11:21:54Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",None,None
...,...,...,...,...,...,...
8851,709074,PractitionerRole/709074,2025-06-16T11:05:19Z,"{""resourceType"":""PractitionerRole"",""active"":tr...",None,None
8852,709075,PractitionerRole/709075,2025-06-16T11:22:27Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",None,None
8853,709076,PractitionerRole/709076,2025-06-16T11:05:19Z,"{""resourceType"":""PractitionerRole"",""active"":tr...",None,None
8854,709077,PractitionerRole/709077,2025-06-16T11:05:19Z,"{""resourceType"":""PractitionerRole"",""active"":tr...",None,None


In [248]:
for role in range(0, len(df)):
    resourceStr = df.iloc[role]['ResourceString']
    roleJSON = json.loads(resourceStr)
    key = roleJSON['practitioner']['identifier']
    #print("The key is ({})".format(key))
    system = key['system']
    value = key['value']
    if (system == 'https://fhir.hl7.org.uk/Id/gmp-number'):
        print(value)
        df.loc[role,'GMP'] = value
    key = roleJSON['organization']['identifier']
    #print("The key is ({})".format(key))
    system = key['system']
    value = key['value']
    if (system == 'https://fhir.nhs.uk/Id/ods-organization-code'):
        print(value)
        df.loc[role,'ODS'] = value

practitionerroles = pd.merge(practitionerroles, df, how="left", on=["ODS","GMP"]).set_index('GMP')
## Make a copy of the index as a column
practitionerroles['GMP'] = practitionerroles.index.astype(str)
practitionerroles['_id'] = practitionerroles['_id'].fillna(-1).astype(int)
practitionerroles

G0108324
A81001
G0109662
A83070
G0110134
B81013
G0229212
A86030
G0229841
A85016
G0230557
A81005
G0230856
A83040
G0231015
A81007
G0232298
A84007
G0233172
B81014
G0320450
A89029
G0322270
A84009
G0322438
A85005
G0324777
B81032
G0325118
A89006
G0325905
A81016
G0326683
A88003
G0328094
A86018
G0416490
A85017
G0417013
B81603
G0417666
A86010
G0417848
A86021
G0419077
A85004
G0507992
B81021
G0605711
A86018
G0610452
A86016
G0611271
A85002
G0611721
A83018
G0611817
A87007
G0613778
A89009
G0613895
A86015
G0715319
A82020
G0719234
A82041
G0719492
A88006
G0822251
A85007
G0822907
A85009
G0823324
A84045
G0824064
B81035
G0824150
B81606
G0826626
A81032
G0827634
A86004
G0828563
A82044
G1010589
A82035
G1011700
A85010
G1011755
A82033
G1110331
A84021
G1213090
A83018
G1215597
A81027
G1215975
A86033
G1216653
A81007
G1216811
A83021
G1217568
A82032
G1326251
A89013
G1328473
A85601
G1330829
B81068
G1338360
A81060
G1339220
A87012
G1340761
A81006
G1340912
A81036
G1341755
A87004
G1341906
B81082
G1342165
A82007
G1342268

,_id_x,Practitioner_Name,ODS,Started,Ended,_id_y,Organisation_Name,_id,Key,_lastUpdated,ResourceString,GMP
GMP,,,,,,,,,,,,
G0102926,680161,ANDERSON MG,D81001,19740401,20250514.0,683460,LENSFIELD MEDICAL PRACTICE,690318,PractitionerRole/690318,2025-06-16T11:21:53Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",G0102926
G0105912,680162,ADLER S,E83600,19740401,20250514.0,684000,ADLER JS-THE SURGERY,690319,PractitionerRole/690319,2025-06-16T11:21:53Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",G0105912
G0107031,680163,ATTWOOD DC,P82013,19740401,20250514.0,695388,LEVER CHAMBERS 2,702431,PractitionerRole/702431,2025-06-16T11:21:54Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",G0107031
G0108018,680173,ALLDRIDGE DGE,H81083,19740401,20250514.0,685936,MOAT HOUSE SURGERY,690320,PractitionerRole/690320,2025-06-16T11:21:54Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",G0108018
G0108324,680174,ANDERSON CF,A81001,19740401,20250514.0,680154,THE DENSHAM SURGERY,682182,PractitionerRole/682182,2025-06-16T11:21:54Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",G0108324
...,...,...,...,...,...,...,...,...,...,...,...,...
G3430202,702425,MATHEWS BH,H81072,19740401,NaN,685925,MEDWYN SURGERY,709074,PractitionerRole/709074,2025-06-16T11:05:19Z,"{""resourceType"":""PractitionerRole"",""active"":tr...",G3430202
G3430226,702426,ALLISON GE,E81023,19740401,20250514.0,683777,CATER STREET SURGERY,709075,PractitionerRole/709075,2025-06-16T11:22:27Z,"{""resourceType"":""PractitionerRole"",""active"":fa...",G3430226
G3430264,702427,LAWTON SL,B86109,19740401,NaN,682628,KIRKSTALL LANE MEDICAL CENTRE,709076,PractitionerRole/709076,2025-06-16T11:05:19Z,"{""resourceType"":""PractitionerRole"",""active"":tr...",G3430264


In [249]:
from fhir.resources.R4B.practitionerrole import PractitionerRole

def convertPractitionerRoleFHIR(GMP):

    practitionerRoleJSON = {
        "resourceType": "PractitionerRole",
        "active": True,
        "practitioner": {
            "identifier": {
                "system": "https://fhir.hl7.org.uk/Id/gmp-number",
                "value": GMP
            },
            "display": practitionerroles.loc[GMP,'Practitioner_Name']
        },
        "organization": {
            "identifier": {
                "system": "https://fhir.nhs.uk/Id/ods-organization-code",
                "value": practitionerroles.loc[GMP,'ODS']
            },
            "display": practitionerroles.loc[GMP,'Organisation_Name']
        },
        "code": [
            {
                "coding": [
                    {
                        "system": "http://snomed.info/sct",
                        "code": "62247001",
                        "display": "General practitioner"
                    }
                ]
            }
        ],
        "specialty": [
            {
                "coding": [
                    {
                        "system": "http://snomed.info/sct",
                        "code": "394814009",
                        "display": "General practice (specialty) (qualifier value)"
                    }
                ]
            }
        ]
    }
    if practitionerroles.loc[GMP,'_id_x'] != '' and not pd.isnull(practitionerroles.loc[GMP,'_id_x']) :
        practitionerRoleJSON['practitioner']['reference'] = 'Practitioner/'+str(practitionerroles.loc[GMP,'_id_x'])
    if practitionerroles.loc[GMP,'_id_y'] != '' and not pd.isnull(practitionerroles.loc[GMP,'_id_y']) :
        practitionerRoleJSON['organization']['reference'] = 'Organization/'+str(practitionerroles.loc[GMP,'_id_y'])
    if practitionerroles.loc[GMP,'Ended'] != '' and not pd.isnull(practitionerroles.loc[GMP,'Ended']) :
        practitionerRoleJSON['active'] = False
    # validate JSON
    practitionerRole = PractitionerRole(**practitionerRoleJSON)

    return practitionerRoleJSON

In [250]:
import json

headers = {"Content-Type": "application/fhir+json"}
fhirurl = "http://localhost:32783/fhir/r4"

new = practitionerroles[['GMP']].head(maxentries).copy()
for GMP in new.index:

    practitionerRoleJSON = convertPractitionerRoleFHIR(GMP)
    if (practitionerroles.loc[GMP,'_id'] == -1):
        print(json.dumps(practitionerRoleJSON, indent=2, ensure_ascii=False))
        # Create

        r = requests.post(fhirurl+'/PractitionerRole', data=json.dumps(practitionerRoleJSON), headers=headers)

        if 'Location' in r.headers:
            location = r.headers['Location'].split('PractitionerRole/')[1].split('/')[0]
            #practitioners.loc[GMP,'_id'] = str(location)
            print('Created Role ' + GMP + ' id - '+location)
        else:
            print("No Location header in response: ",r.status_code)
            print("Response headers:", r.headers)
            print(json.dumps(practitionerRoleJSON, indent=2, ensure_ascii=False))
            print(r.text)
    else:
        # Update

        isUpdate = False
        if practitionerroles.loc[GMP,'ResourceString'] != '':
            practitionerroleMDM = json.loads(practitionerroles.loc[GMP,'ResourceString'])
            if practitionerroleMDM['active'] != practitionerRoleJSON['active']:
                isUpdate = True
                practitionerroleMDM['active'] = practitionerRoleJSON['active']
                print('active')

            if isUpdate:
                #print(json.dumps(organisationMDM, indent=2, ensure_ascii=False))
                r = requests.put(fhirurl+'/PractitionerRole/'+practitionerroleMDM['id'], data=json.dumps(practitionerroleMDM), headers=headers)

                if 'Location' in r.headers:
                    location = r.headers['Location'].split('PractitionerRole/')[1].split('/')[0]
                    #organisations.loc[org,'ResourceString'] = json.dumps(organisationMDM)
                    print('Updated Role ' + GMP + ' id - '+location)
                else:
                    print("No Location header in response: ",r.status_code)
                    print("Response headers:", r.headers)
                    print(json.dumps(practitionerroleMDM, indent=2, ensure_ascii=False))
                    print(r.text)


## Testing FHIR (Validation)

So far we have just created FHIR resources as JSON. We have performed basic schema validation using a [fhir.resources](https://github.com/nazrulworld/fhir.resources). Note this package uses FHIR R4B, not R4 and we are using R4 - confused, none of the resources in FHIR R4 changed in R4B, so this is fine.

You can also validate FHIR using command line tools such as [FHIR CLI Validator](https://confluence.hl7.org/spaces/HAFWG/pages/248876078/Using+the+FHIR+Validator+Locally+Quick+Guide) or online applications such as [validate.fhir.org](https://validator.fhir.org/).

Note these tools will generate warnings around England content; you can reduce these warnings by using the [NHS England UK Core](https://digital.nhs.uk/services/fhir-uk-core) package. We use our own package [Virtual Healthcare Testing](https://virtually-healthcare.github.io/R4/testing.html) which incorporates UK Core and extra NHS England data requirements. Documentation on Virtually Healthcare data requirements can be found below, these are called FHIR Profiles:

- [Organization](https://virtually-healthcare.github.io/R4/StructureDefinition-Organization.html)
- [Practitioner](https://virtually-healthcare.github.io/R4/StructureDefinition-Practitioner.html)
- [PractitionerRole](https://virtually-healthcare.github.io/R4/StructureDefinition-PractitionerRole.html)

The profiles are stricter than UK Core as these need to be followed in several products, they are generally conformant to wider NHS England data requirements (not just FHIR).

## Working with a FHIR Test Server

How to put the resources we built earlier into a FHIR Server is available on the internet, and so we won't repeat that.

If you wish to experiment with this, I would suggest using the [HAPI FHIR Test Server](https://hapi.fhir.org/). E.g.

`POST http://hapi.fhir.org/baseR4/Organization`

`POST http://hapi.fhir.org/baseR4/Practitioner`

`POST http://hapi.fhir.org/baseR4/PractitionerRole`

Once you have added the resources to HAPI FHIR, you should be able to search for them, e.g.

`GET http://hapi.fhir.org/baseR4/Organization?identifier=https://fhir.nhs.uk/Id/ods-organization-code|F83004`

`GET http://hapi.fhir.org/baseR4/Practitioner?identifier=https://fhir.hl7.org.uk/Id/gmp-number|G3298457

## Practical Implementation

So far we have a relatively simple model for our GPs and Practices both are strongly identified using national identifiers, but in practice we will have several other identifiers. Existing use of these national identifiers may not be robust and have data issues. This can occur in all EPR systems, including secondary care.

The main issue is although GMP is defined [GENERAL MEDICAL PRACTITIONER PPD CODE](https://www.datadictionary.nhs.uk/attributes/general_medical_practitioner_ppd_code.html) this and the other practitioner identifiers are quite frequently mixed up.

How to handle this is beyond the scope of this walkthrough, a list of all the different practitioner identifiers can be found on [NHS North West GMSA](https://nw-gmsa.github.io/R4/StructureDefinition-EnglandPractitionerIdentifier.html)

Many systems will have their own strong identifier — for example, EMIS uses UUID's to identify practitioners across all its API's. Our use case is master data management, so it makes sense for us to have a record of that in our MDM solution. As suppliers are supporting operational delivery of care and that ODS is only updated quarterly (and monthly), it's likely that our Practitioner may have more details than ODS or is more up to date.

This means we need to cope with existing data, our data load needs to be repeatable (so we can schedule quarterly/monthly) runs and we can merge with existing data.